In [1]:
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as D
from torch.utils.data import DataLoader, TensorDataset

n = 5
epochs = 5
supervisionEpochs = 10
lr = 0.001
log_interval = 20
trainSize = 100000
penaltyLambda = 10
doublePeakHighMean = 0.9
doublePeakLowMean = 0.1
doublePeakStd = 0.1

d1 = D.normal.Normal(doublePeakLowMean, doublePeakStd)
d2 = D.normal.Normal(doublePeakHighMean, doublePeakStd)
distributionRatio = (d1.cdf(1) + d2.cdf(1) - d1.cdf(0) - d2.cdf(0)) / 2
distributionBase = d1.cdf(0) + d2.cdf(0)


def cdf(x, i=None):
    return (d1.cdf(x) + d2.cdf(x) - distributionBase) / 2 / distributionRatio

# def cdf(x, i=None):
#     if x < 0.1:
#         return 0
#     if x <= 0.2:
#         return 0.5 * (x - 0.1) / 0.1
#     if x < 0.8:
#         return 0.5
#     if x < 0.9:
#         return 0.5 + 0.5 * (x - 0.8) / 0.1
#     return 1


print(distributionBase)

tensor(0.1587)


In [2]:
signals = np.random.randint(2, size=(trainSize, n))
# samples1 = np.random.uniform(low=0.1, high=0.2, size=(trainSize, n))
# samples2 = np.random.uniform(low=0.8, high=0.9, size=(trainSize, n))
samples1 = np.random.normal(
    loc=doublePeakLowMean, scale=doublePeakStd, size=(trainSize, n)
)
for i in range(trainSize):
    for j in range(n):
        while samples1[i, j] < 0 or samples1[i, j] > 1:
            samples1[i, j] = np.random.normal(
                loc=doublePeakLowMean, scale=doublePeakStd
            )
samples2 = np.random.normal(
    loc=doublePeakHighMean, scale=doublePeakStd, size=(trainSize, n)
)
for i in range(trainSize):
    for j in range(n):
        while samples2[i, j] < 0 or samples2[i, j] > 1:
            samples2[i, j] = np.random.normal(
                loc=doublePeakHighMean, scale=doublePeakStd
            )
samplesJoint = signals * samples1 - (signals - 1) * samples2
tp_tensor = torch.tensor(samplesJoint, dtype=torch.float32)
# tp_tensor = torch.tensor(np.random.rand(10000, n), dtype=torch.float32)
tp_dataset = TensorDataset(tp_tensor[: round(trainSize * 0.3)])
tp_dataset_testing = TensorDataset(tp_tensor[round(trainSize * 0.7) :])
tp_dataloader = DataLoader(tp_dataset, batch_size=128, shuffle=True)
tp_dataloader_testing = DataLoader(tp_dataset_testing, batch_size=256, shuffle=False)


# for mapping binary to payments before softmax
model = nn.Sequential(
    nn.Linear(n, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, n),
)


# optimizer = optim.SGD(model.parameters(), lr=lr)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [3]:
def bitsToPayments(bits):
    if sum(bits).item() == 0:
        return torch.ones(n)
    bits = bits.type(torch.float32)
    negBits = torch.ones(n) - bits
    payments = model(bits)
    payments = payments - 1000 * negBits
    payments = torch.softmax(payments, 0)
    payments = payments + negBits
    return payments


def tpToBits(tp, bits=torch.ones(n).type(torch.uint8)):
    payments = bitsToPayments(bits)
    newBits = (tp >= payments).type(torch.uint8)
    
    if torch.equal(newBits, bits):
        return bits
    else:
        return tpToBits(tp, newBits)#bits-bits#tpToBits(tp, newBits)


def tpToPayments(tp):
    return bitsToPayments(tpToBits(tp))


def tpToTotalDelay(tp):
    return n - sum(tpToBits(tp).type(torch.float32))


In [4]:
print(1_000_000)
print(cdf(0.5))

1000000
tensor(0.5000)


In [5]:
dpPrecision = 100
# howManyPpl left, money left, yes already
dp = np.zeros([n + 1, dpPrecision + 1, n + 1])
decision = np.zeros([n + 1, dpPrecision + 1, n + 1], dtype=np.uint8)
# ppl = 0 left
for yes in range(n + 1):
    for money in range(dpPrecision + 1):
        if money == 0:
            dp[0, 0, yes] = 0
        else:
            dp[0, money, yes] = yes# + 1.0
for ppl in range(1, n + 1):
    for yes in range(n + 1):
        for money in range(dpPrecision + 1):
            minSoFar = 1_000_000
            for offerIndex in range(money + 1):
                offer = offerIndex / dpPrecision
                res = (1 - cdf(offer)) * dp[
                    ppl - 1, money - offerIndex, min(yes + 1, n)
                ] + cdf(offer) * (1 + dp[ppl - 1, money, yes])
                if minSoFar > res:
                    minSoFar = res
                    decision[ppl, money, yes] = offerIndex
            dp[ppl, money, yes] = minSoFar


In [6]:
print(dp[n, dpPrecision, 0])

1.8651759624481201


In [7]:
# howManyPpl left, money left, yes already
dp_H = np.zeros([n + 1 , n + 1, dpPrecision + 1])
decision_H = np.zeros([n + 1 , n + 1, dpPrecision + 1], dtype=np.uint8)
# ppl = 0 left
for i in range(1,n+1):
    for money in range(dpPrecision + 1):
        if money == 0:
            dp_H[i, 0, 0] = 1
        else:
            offer = money / dpPrecision
            dp_H[i, 0, money] = 0#cdf(offer)# + 1.0
for i in range(1,n+1):
    for ppl in range(1, i + 1):
        for money in range(dpPrecision + 1):
            maxSoFar = -1_000_000
            for offerIndex in range(money + 1):
                offer = offerIndex / dpPrecision
                res = (1-cdf(offer)) * dp_H[
                     i, ppl - 1, money - offerIndex
                    ]
                if maxSoFar < res:
                    maxSoFar = res
                    decision_H[i, ppl, money] = offerIndex
            dp_H[i, ppl, money] = maxSoFar
    print("\n",i)
    print(dp_H[i, i, dpPrecision])
    print(5*(1-dp_H[i, i, dpPrecision]))


 1
0.0
5.0

 2
0.2500000596046448
3.749999701976776

 3
0.2585742771625519
3.7071286141872406

 4
0.27404484152793884
3.629775792360306

 5
0.28367453813552856
3.581627309322357


In [8]:
print(samplesJoint)

[[0.2119784  0.00957109 0.87493958 0.9531459  0.25366211]
 [0.92031721 0.08410549 0.80713045 0.81872626 0.09812399]
 [0.79623836 0.98324441 0.05464513 0.22718182 0.28997579]
 ...
 [0.89845756 0.06159976 0.9991427  0.95588083 0.84658061]
 [0.1142967  0.23687651 0.88480151 0.73002182 0.86287422]
 [0.85125635 0.10518473 0.17213499 0.09106445 0.20469796]]


In [9]:
def plan_dp(temp):
    #print(temp)
    remain=dpPrecision
    yes=0;
    ans =0;
    o_list=[];
    remain_list=[];
    for ppl in range(n,0,-1):
        o=decision[ppl, remain, yes]
        #print(o,remain)
        o_list.append(o)
        remain_list.append(remain);
        if(o<temp[n-ppl]):
            remain-=int(o);
            yes+=1;
        elif (remain>0):
            ans+=1;
    if(remain<=0):
        return ans,o_list;
    else:
        return n,o_list;
    

In [10]:
ans_list=[];
for i in range(10000):
    temp=samplesJoint[i]*dpPrecision
    #print(temp)
    ans_list.append(plan_dp(temp)[0]);
    #print("\n",temp)
    #print(plan_dp(temp))
print(sum(ans_list)/len(ans_list))

1.8411


In [11]:
def dpSupervisionRule(tp):
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0 for ii in range(n)]
    money = dpPrecision
    yes = 0
    for i in range(n):
        offerIndex = decision[n - i, money, yes]
        offer = offerIndex / dpPrecision
        if tp[i] >= offer:
            money -= offerIndex
            yes += 1
            bits[i] = 1
            payments[i] = offer
        else:
            bits[i] = 0
            payments[i] = 1
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    # print()
    # print(tp)
    # print(bits)
    # print(payments)
    # print()
    return (bits, payments)


def heuristicSupervisionRule(tp):
    tp = list(tp.numpy())
    bits = [1 for ii in range(n)]
    payments = [0 for ii in range(n)]

    for turns in range(n,0,-1):
        money = dpPrecision
        j=0
        tempo=sum(bits)
        #print("bits",tempo)
        for i in range(n):
            if(j>=n):
                break;
            
            offerIndex = decision_H[tempo ,tempo - i, money]
            offer = offerIndex / dpPrecision
            while(j<n):
                if(bits[j]!=0):
                    break;
                j+=1;
            if(j>=n):
                break;
            if tp[j] >= offer:
                #print(money,j,tp[j],offer)
                money -= offerIndex
                bits[j] = 1
                payments[j] = offer
            else:
                bits[j] = 0;
                payments[j] = 1;
            j+=1;
        #print("money",money)
        if(money==0):
            break;
    if money > 0:
        bits = [0 for ii in range(n)]
        payments = [1 for ii in range(n)]

    bits = torch.tensor(bits, dtype=torch.float32)
    payments = torch.tensor(payments, dtype=torch.float32)
    # print()
    # print(tp)
    # print(bits)
    # print(payments)
    # print()
    return (bits, payments)

for i in range(50):
    temp=torch.tensor(samplesJoint[i])
    #print(temp)
    print(temp);
    print(heuristicSupervisionRule(temp))
    print()
    
def dpDelay(tp):
    bits, payments = dpSupervisionRule(tp)
    totalPayment = torch.dot(bits.type(torch.float32), payments).item()
    if totalPayment < 1:
        return n
    else:
        return n - sum(bits).item()


def costSharingSupervisionRule(tp):
    tp = list(tp.numpy())
    for k in range(n, -1, -1):
        if k == 0:
            break
        bits = [1 if tp[ii] >= 1 / k else 0 for ii in range(n)]
        if sum(bits) == k:
            break
    if k == 0:
        payments = [1 for ii in range(n)]
    else:
        payments = [1 / k if bits[ii] == 1 else 1 for ii in range(n)]
    bits = torch.tensor(bits, dtype=torch.uint8)
    payments = torch.tensor(payments, dtype=torch.float32)
    return (bits, payments)


def costSharingDelay(tp):
    return n - sum(costSharingSupervisionRule(tp)[0]).item()

def heuristicDelay(tp):
    return n - sum(heuristicSupervisionRule(tp)[0]).item()

tensor([0.2120, 0.0096, 0.8749, 0.9531, 0.2537], dtype=torch.float64)
(tensor([0., 0., 1., 1., 0.]), tensor([1.0000, 1.0000, 0.4700, 0.5300, 1.0000]))

tensor([0.9203, 0.0841, 0.8071, 0.8187, 0.0981], dtype=torch.float64)
(tensor([1., 1., 1., 1., 0.]), tensor([0.0800, 0.0700, 0.0700, 0.7800, 1.0000]))

tensor([0.7962, 0.9832, 0.0546, 0.2272, 0.2900], dtype=torch.float64)
(tensor([1., 1., 0., 1., 0.]), tensor([0.7900, 0.1000, 1.0000, 0.1100, 1.0000]))

tensor([0.0975, 0.7604, 0.8710, 0.9742, 0.0629], dtype=torch.float64)
(tensor([1., 1., 1., 1., 0.]), tensor([0.0800, 0.0700, 0.0700, 0.7800, 1.0000]))

tensor([0.9632, 0.1744, 0.0678, 0.9455, 0.1016], dtype=torch.float64)
(tensor([1., 1., 0., 1., 0.]), tensor([0.0800, 0.0700, 1.0000, 0.8500, 1.0000]))

tensor([0.9211, 0.0462, 0.8134, 0.9593, 0.7966], dtype=torch.float64)
(tensor([1., 0., 1., 1., 1.]), tensor([0.0500, 1.0000, 0.0900, 0.0800, 0.7800]))

tensor([0.9731, 0.1126, 0.9696, 0.8958, 0.1014], dtype=torch.float64)
(tensor([1., 1., 1

In [12]:
for i in range(n, 0, -1):
            print([1 if ii >= i else 0 for ii in range(n)])
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )

[0, 0, 0, 0, 0]
tensor([0.1897, 0.2278, 0.2114, 0.1784, 0.1927], grad_fn=<AddBackward0>)
[0, 0, 0, 0, 1]
tensor([0.2339, 0.2793, 0.2621, 0.2247, 1.0000], grad_fn=<AddBackward0>)
[0, 0, 0, 1, 1]
tensor([0.3081, 0.3578, 0.3341, 1.0000, 1.0000], grad_fn=<AddBackward0>)
[0, 0, 1, 1, 1]
tensor([0.4563, 0.5437, 1.0000, 1.0000, 1.0000], grad_fn=<AddBackward0>)
[0, 1, 1, 1, 1]
tensor([1., 1., 1., 1., 1.], grad_fn=<AddBackward0>)


In [13]:
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]
print(allBits)

for batch_idx, (tp_batch,) in enumerate(tp_dataloader_testing):
    penalty = 0
    for bitsMoreOnes in allBits:
        for i in range(n):
            if bitsMoreOnes[i] == 1:
                bitsLessOnes = bitsMoreOnes.clone()
                bitsLessOnes[i] = 0
                penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
    loss = penalty * penaltyLambda
    for tp in tp_batch:
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                #loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0
                loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0
    print()
    print(tp)
    tp1 = tp.clone()
    tp1[0] = 1
    tp0 = tp.clone()
    tp0[0] = 0
    offer = tpToPayments(tp1)[0]
    print(offer)
    print(tpToPayments(tp1))
    print(delay1)
    print(delay0)
    break;
#print(loss)
#print(penalty)

[tensor([0, 0, 0, 0, 0]), tensor([0, 0, 0, 0, 1]), tensor([0, 0, 0, 1, 0]), tensor([0, 0, 0, 1, 1]), tensor([0, 0, 1, 0, 0]), tensor([0, 0, 1, 0, 1]), tensor([0, 0, 1, 1, 0]), tensor([0, 0, 1, 1, 1]), tensor([0, 1, 0, 0, 0]), tensor([0, 1, 0, 0, 1]), tensor([0, 1, 0, 1, 0]), tensor([0, 1, 0, 1, 1]), tensor([0, 1, 1, 0, 0]), tensor([0, 1, 1, 0, 1]), tensor([0, 1, 1, 1, 0]), tensor([0, 1, 1, 1, 1]), tensor([1, 0, 0, 0, 0]), tensor([1, 0, 0, 0, 1]), tensor([1, 0, 0, 1, 0]), tensor([1, 0, 0, 1, 1]), tensor([1, 0, 1, 0, 0]), tensor([1, 0, 1, 0, 1]), tensor([1, 0, 1, 1, 0]), tensor([1, 0, 1, 1, 1]), tensor([1, 1, 0, 0, 0]), tensor([1, 1, 0, 0, 1]), tensor([1, 1, 0, 1, 0]), tensor([1, 1, 0, 1, 1]), tensor([1, 1, 1, 0, 0]), tensor([1, 1, 1, 0, 1]), tensor([1, 1, 1, 1, 0]), tensor([1, 1, 1, 1, 1])]

tensor([0.2642, 0.1255, 0.0803, 0.2120, 0.0543])
tensor(1., grad_fn=<SelectBackward>)
tensor([1., 1., 1., 1., 1.], grad_fn=<AddBackward0>)
tensor(4.)
tensor(5.)


In [14]:
allBits = [torch.tensor(bits) for bits in itertools.product([0, 1], repeat=n)]

runningLossNN = []
runningLossCS = []
runningLossDP = []
runningLossHeuristic = []

def recordAndReport(name, source, loss, n=100):
    source.append(loss)
    realLength = min(n, len(source))
    avgLoss = sum(source[-n:]) / realLength
    print(f"{name} ({realLength}): {avgLoss}")


def supervisionTrain(epoch, supervisionRule):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        loss = 0
        for tp in tp_batch:
            bits, payments = supervisionRule(tp)
            # print()
            # print("supervision")
            # print(tp)
            # print(bits)
            # print()
            # print(payments)
            # print(bitsToPayments(bits))
            # print()
            loss = loss + F.mse_loss(bitsToPayments(bits), payments)

        loss = loss / len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            #print()
            #print(tp)
            #print(bits)
            #print(payments)
            #print(bitsToPayments(bits))
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )


def train(epoch):
    model.train()
    for batch_idx, (tp_batch,) in enumerate(tp_dataloader):
        optimizer.zero_grad()
        penalty = 0
        for bitsMoreOnes in allBits:
            for i in range(n):
                if bitsMoreOnes[i] == 1:
                    bitsLessOnes = bitsMoreOnes.clone()
                    bitsLessOnes[i] = 0
                    penalty = penalty + sum(
                        torch.relu(
                            bitsToPayments(bitsMoreOnes) - bitsToPayments(bitsLessOnes)
                        )
                    )
        loss = penalty * penaltyLambda
        # costSharingLoss = 0
        # dpLoss = 0
        for tp in tp_batch:
            # costSharingLoss += costSharingDelay(tp)
            # dpLoss += dpDelay(tp)
            # print()
            # print("---")
            # print(tp)
            # print(costSharingSupervisionRule(tp))
            # print(dpSupervisionRule(tp))
            # print(costSharingDelay(tp), dpDelay(tp))
            # print()
            for i in range(n):
                tp1 = tp.clone()
                tp1[i] = 1
                tp0 = tp.clone()
                tp0[i] = 0
                offer = tpToPayments(tp1)[i]
                delay1 = tpToTotalDelay(tp1)
                delay0 = tpToTotalDelay(tp0)
                loss = loss + (1 - cdf(offer)) * delay1 + cdf(offer) * delay0

        loss = loss / len(tp_batch) / n
        # costSharingLoss /= len(tp_batch)
        # dpLoss /= len(tp_batch)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(tp_batch),
                    len(tp_dataloader.dataset),
                    100.0 * batch_idx / len(tp_dataloader),
                    loss.item(),
                )
            )
            # recordAndReport("NN", runningLossNN, loss.item())
            # recordAndReport("CS", runningLossCS, costSharingLoss)
            # recordAndReport("DP", runningLossDP, dpLoss)
            # print(dp[n, dpPrecision, 0])
            print("penalty:",penalty.item())
            # print(distributionRatio)
            # for i in range(n, 0, -1):
            #     print(
            #         tpToPayments(
            #             torch.tensor([1] * i + [0] * (n - i), dtype=torch.float32)
            #         )
            #     )


In [15]:
def test():
    model.eval()
    with torch.no_grad():
        costSharingLoss = 0
        dpLoss = 0
        nnLoss = 0
        heuristicLoss=0
        lenLoss= 0
        for (tp_batch,) in tp_dataloader_testing:

            for tp in tp_batch:
                costSharingLoss += costSharingDelay(tp)
                dpLoss += dpDelay(tp)
                nnLoss += tpToTotalDelay(tp)
                heuristicLoss+= heuristicDelay(tp)
            lenLoss+=len(tp_batch)
        costSharingLoss /= lenLoss
        dpLoss /= lenLoss
        nnLoss /= lenLoss
        heuristicLoss /= lenLoss
        print(lenLoss)
        recordAndReport("NN", runningLossNN, nnLoss)
        recordAndReport("CS", runningLossCS, costSharingLoss)
        recordAndReport("DP", runningLossDP, dpLoss)
        recordAndReport("heuristic", runningLossHeuristic, heuristicLoss)
        print("DP:",dp[n, dpPrecision,0])
        for i in range(n, 0, -1):
            print("Heuristic:",i,5*(1-dp_H[i, i, dpPrecision]))
        for i in range(n, 0, -1):
            print(
                    tpToPayments(
                            torch.tensor([0 if ii >= i else 1 for ii in range(n)], dtype=torch.float32)
                    )
                )

#model=torch.load("save/pytorchNN=5dp1");
#model.eval()
for epoch in range(1, supervisionEpochs + 1):
    print(distributionRatio)
#    #supervisionTrain(epoch, costSharingSupervisionRule)
#    supervisionTrain(epoch, dpSupervisionRule)
    supervisionTrain(epoch, heuristicSupervisionRule)
test()

for epoch in range(1, epochs + 1):
    train(epoch)
    test()


tensor(0.8413)
Train Epoch: 1 [0/30000 (0%)]	Loss: 0.059296
Train Epoch: 1 [2560/30000 (9%)]	Loss: 0.017876
Train Epoch: 1 [5120/30000 (17%)]	Loss: 0.014282
Train Epoch: 1 [7680/30000 (26%)]	Loss: 0.008371
Train Epoch: 1 [10240/30000 (34%)]	Loss: 0.004184
Train Epoch: 1 [12800/30000 (43%)]	Loss: 0.005141
Train Epoch: 1 [15360/30000 (51%)]	Loss: 0.004757
Train Epoch: 1 [17920/30000 (60%)]	Loss: 0.003990
Train Epoch: 1 [20480/30000 (68%)]	Loss: 0.003672
Train Epoch: 1 [23040/30000 (77%)]	Loss: 0.001296
Train Epoch: 1 [25600/30000 (85%)]	Loss: 0.002891
Train Epoch: 1 [28160/30000 (94%)]	Loss: 0.000619
tensor(0.8413)
Train Epoch: 2 [0/30000 (0%)]	Loss: 0.002349
Train Epoch: 2 [2560/30000 (9%)]	Loss: 0.000379
Train Epoch: 2 [5120/30000 (17%)]	Loss: 0.001822
Train Epoch: 2 [7680/30000 (26%)]	Loss: 0.000356
Train Epoch: 2 [10240/30000 (34%)]	Loss: 0.001348
Train Epoch: 2 [12800/30000 (43%)]	Loss: 0.002295
Train Epoch: 2 [15360/30000 (51%)]	Loss: 0.000710
Train Epoch: 2 [17920/30000 (60%)]	Los

Train Epoch: 2 [7680/30000 (26%)]	Loss: 1.801340
penalty: 0.0
Train Epoch: 2 [10240/30000 (34%)]	Loss: 1.635245
penalty: 0.0
Train Epoch: 2 [12800/30000 (43%)]	Loss: 1.760734
penalty: 0.0
Train Epoch: 2 [15360/30000 (51%)]	Loss: 1.680132
penalty: 0.009101510047912598
Train Epoch: 2 [17920/30000 (60%)]	Loss: 1.660180
penalty: 0.0
Train Epoch: 2 [20480/30000 (68%)]	Loss: 1.825090
penalty: 0.0
Train Epoch: 2 [23040/30000 (77%)]	Loss: 1.759072
penalty: 0.0058272480964660645
Train Epoch: 2 [25600/30000 (85%)]	Loss: 1.660376
penalty: 0.0
Train Epoch: 2 [28160/30000 (94%)]	Loss: 1.661407
penalty: 0.0
30000
NN (3): 1.7835556268692017
CS (3): 2.5372333333333335
DP (3): 1.8787333333333336
heuristic (3): 1.8137333333333334
DP: 1.8651759624481201
Heuristic: 5 3.581627309322357
Heuristic: 4 3.629775792360306
Heuristic: 3 3.7071286141872406
Heuristic: 2 3.749999701976776
Heuristic: 1 5.0
tensor([0.0663, 0.0642, 0.0639, 0.0633, 0.7423])
tensor([0.0894, 0.0820, 0.0855, 0.7431, 1.0000])
tensor([0.7713,

In [16]:

torch.save(model, "save/pytorchNN=5heuristic2")

In [17]:
#model = torch.load(PATH)
#model.eval()